# Part 4: The Analog-to-Digital Converter (ADC)


#### Learning Goals
* How a successive approximation register (SAR) ADC works
* Establish familiarity with the ADC configuration options available on an STM32
* Use of interrupts to handle sampled data in a program

#### Support Literature
* [Supplement on Successive Approximation ADCs](Documents/SAR_supplement.pdf)
* [User manual STM32 Nucleo-64 boards](https://www.st.com/content/ccc/resource/technical/document/user_manual/98/2e/fa/4b/e0/82/43/b7/DM00105823.pdf/files/DM00105823.pdf/jcr:content/translations/en.DM00105823.pdf)
* [HAL documentation](Documents/um1725-description-of-stm32f4-hal-and-lowlayer-drivers-stmicroelectronics.pdf)
* [Reference Manual stm32f446xx](Documents/dm00135183-stm32f446xx-advanced-arm-based-32-bit-mcus-stmicroelectronics.pdf)

#### Introduction to the SAR ADC

In this task we will learn about how the built-in ADCs on a STM32 F446re Nucleo Board works, and how it is configured to exercise certain behaviours. There are three ADCs available to us, and all of them are [successive approximation register adcs (SAR)](https://en.wikipedia.org/wiki/Successive-approximation_ADC). Sampling with a SAR ADC is controlled by a clock signal, and happens in two stages:
1. The input voltage is fed to a "sample & hold" circuit, which functions as a capacitor and switch. In the "sample" phase of, the capacitor is connected to the signal input, and charges to track the input voltage. Once a certain number of clock cycles are completed, the input line is disconnected from the capacitor, leaving the current stored in  the capacitor with nowhere to go. The voltage over the capacitor will now remain constant, while the successive approximation register performs the conversion.

![](Figures/sample_hold.png)

2. The successive approximation register converts the capacitor voltage to a binary number using something akin to [binary search](https://en.wikipedia.org/wiki/Binary_search) to "home in" on the precise capacitor voltage. The circuit will be able to do 1 bisection step (i.e. generating 1 bit of the ADC output value) per clock cycle, meaning a 12 bit A/D conversion requires at minimum 12 clock cycles. Then comes some overhead for clearing registers between conversions etc. [**Here is an animated illustration**](Figures/SAR.gif) showing a 6-bit SAR performing a single conversion.

<!--![](Figures/ADC_block_diagram.png)-->


#### SAR conversion time

A limiting factor for precisely how fast a signal may be sampled is the time it takes to complete one conversion. There are a number of parameters influencing the total A/D conversion time, and for a SAR ADC such as the one in the STM32 F446re microcontroller, these paramters are:

<!--For simple systems using analog input without strict requirements to high sampling speed it is usually sufficient to keep the ADC in "single conversion mode", where each A/D conversion is initiated by a function call (such as `analogRead()` on an arduino). However, it is also possible to configure the ADC to begin a new A/D conversion immediately after completing the previous one. In this case, the sampling frequency will be determined by a number of factors which will vary depending on the type of ADC. For a Successive Approximation ADC such as the one in a STM32 F446RE, these factors are:-->

* **Peripheral Clock Signal**: A square wave clock signal generated by prescaling the system clock by some integer factor. Controls the working speed of a number of peripheral components, including the ADC. Genera
* **ADC Clock Prescaler**: An integer factor which determines the clock speed of the ADC relative to peripheral clock.
    * Example: If the peripheral clock is $84 \text{MHz}$, and the ADC clock prescaler is $8$, the ADC clock signal will be $\frac{84\text{ MHz}}{8} = 10.5 \text{MHz}$
* **Sample time:** Number of ADC clock cycles to wait while charging up the capacitor. This amounts to the duration of the "sample" phase of A/D converison.
    * Example: If the sample time is $3$ clock cycles, and the conversion time is $15$ clock cycles, then the total number of clock cycles spent converting a single sample is $T = 3 + 15 = 18$.
* **Conversion time:** Equivalent to the "hold" phase in A/D converison, and is determined by the ADC resolution (more bits = more time). For the STM32 F446RE a 12-bit conversion (full resolution) requires $15$ clock cycles.
 
## a)
Given the ADC settings specified below, how many microseconds ($\mu \text{s}$) does it take for the ADC to complete $1$ analog-to-digital conversion, and what is the maximum sampling rate we can use when configuring the Hardware Timer?
<!-- Calculate the sampling frequency for an ADC with the following settings, and present the answer in $\text{Hz}$:-->

|Parameter|Value|
|---|---|
|Peripheral Clock| $$84 \text{ MHz}$$|
|ADC Clock Prescaler| $$4$$ |
|Sample Time | $$15\text{ cycles}$$ |
|Conversion Time | $$15 \text{ cycles}$$|


In [ ]:
T_conversion = "?"  # Replace with correct answer
fs_max = "?"  # Replace with correct answer
# WRITE YOUR CODE IN THIS CELL:

In [ ]:
from hashlib import sha1
assert sha1(str(round(T_conversion, 3)).encode('utf-8')+b'7d616').hexdigest() == '288d48b341bc40fd6f86c0617377e84aac3a11b5', 'Wrong answer for T_conversion :('; print('Correct answer for T_conversion :)')
assert sha1(str(round(fs_max, 3)).encode('utf-8')+b'8613a').hexdigest() == '7944d5580bba8bbf0b20c54bf7e2520e848b2d95', 'Wrong answer for fs_max :('; print('Correct answer for fs_max :)')

According to our calculations, a sample rate of $f_s = 10 \text{ KHz}$ should provide ample time to complete a full $12$-bit A/D conversion within the available sampling period. We can now proceed to implement this configuration on the STM32.


## b) Setting up the ADC in interrupt mode

In the STM32CubeIDE, go to the device configuration tool for your project and select the "Pinout & Configuration" tab. Select **ADC1** and enable **IN0**. Once this is done we can configure the ADC. Below is a list of which parameters we want to adjust:
1. Choose the following settings in the "Parameter Settings" tab:

    - Clock Prescaler: **PCLK2 divided by 4**
    - Resolution: **12 bits (15 ADC Clock Cycles)**
    - Data Alignment: **Right Alignment**
    - Continuous Conversion Mode: **Disabled**
    - External Trigger Conversion Source: **Timer 2 Trigger Out event**. *This will enusure A/D conversion is controlled by Timer 2's output.*
    - Rank 1 -> Sampling time: **15 Cycles**
   
   Once done, the parameter settings should look [like this](Figures/ADC_config.png).
   
3. Select the "NVIC Settings" tab and click to enable "ADC1, ADC2 and ADC3 interrupts", [like this](Figures/ADC_interrupt.png).

The ADC will now update it's output register every time a new ADC conversion is completed, as well as send an interrupt request. This interrupt request will trigger the execution of a specific function which we will make use of later.

## c) Configuring the DAC
In order to verify successful sampling of the input signal, we will use the DAC to reconstruct the analog input so it can be viewed on an oscilloscope. Go to "Pinout & Configuration", select **DAC** and enable **OUT1**. Make sure "Output Buffer" is set to "Enable". The configuration menu should look like [this](Figures/DAC_config.PNG).

## d) Adding user code to `main.c`
Click "build" (or save the project) to genrate a code template. Then, open the `main.c` file in the `Core/Src` directory. In this file you will see certain fields marked with comments such as `/* USER CODE BEGIN <category> */` and `/* USER CODE END <category> */`. To add your own code to the template, the code must be added somewhere between these comments, otherwise it will be erased if the code template is rebuilt from a modified device configuration at a later time.

We wish to make the following changes:
1. Add a global unsigned int variable to store current ADC value. Mark as `__IO` (aka. volatile) to safeguard against race conditions.

>```c
>/* USER CODE BEGIN PV */
>__IO uint32_t adc_value; 
>/* USER CODE END PV */
>```

2. Define a function which is executed upon an ADC interrupt request. <br>In layman's terms: *Whenever the ADC reports the completion of another A/D conversion, execute this code immediately.*

>```c
>/* USER CODE BEGIN PFP */
>void HAL_ADC_ConvCpltCallback(ADC_HandleTypeDef* hadc){
>	adc_value = HAL_ADC_GetValue(&hadc1); // Read converted sample value from the ADC and store in adc_value.
>	HAL_DAC_SetValue(&hdac, DAC_CHANNEL_1, DAC_ALIGN_12B_R, adc_value); // Write the value in adc_value to DAC channel 1 using 12 bit resolution.
>}
>/* USER CODE END PFP */
>```
>
> - Important note: interrupt procedures should generally be kept as short as possible, with very few operations.

3. Add code to start up the ADC conversion process, as well as the DAC before the loop in the `main()` function.

>```c
>  /* USER CODE BEGIN 2 */
>  /*
>  existing code from part 3
>  */
>  HAL_Delay(100);  // Insert a short delay between ADC/DAC initialisation and startup.
>  HAL_ADC_Start_IT (&hadc1); // Start up the ADC conversion process with interrupts.
>  HAL_DAC_Start(&hdac, DAC_CHANNEL_1); // Start up the DAC with channel 1 as output.
>  /* USER CODE END 2 */
>```

You can now compile your project, and run the program on the STM32

## e) monitor DAC output
To test our feedthrough DSP system we will connect it to a signal generator and an oscilloscope. 

1. Configure the signal grenerator to produce a $f=500Hz$ sine wave oscillating between $0.0\text{ V}$ and $3.3 \text{ V}$
2. Connect the signal generator output to the pin corresponding to ADC channel IN0. 
3. Connect both the signal generator output and the MCU DAC channel 1 output to the oscilloscope.
4. Adjust the settings on the oscilloscope until you have a clear view of the waveform of both the original analog signal, and the reconstructed DAC output. 
5. Take a screen capture of the oscilloscope display, and paste it into the markdown cell below.




<div class="alert alert-info">
<h4> Answer theory questions here!</h4>
</div>
